This notebook just demonstrates different methods of calculating the PCA and how they behave with different data.

The default data used is the `cars.csv` from the homework. Alternatively there are an arbitrary $4 \times 3$ as well as a $3 \times 2$ matrix provided. For the small matrix we did the calculations by hand to compare them to the results, you can find them below.

The tests are run three times: Once for the raw data, once for the centered data and once for the normalized data (i.e. mean 0 and std 1).

In the plots and text outputs (set `TEXTOUT = True`!) created when you run the cell, the following names are used:

* `eig`: Calculates the covariance matrix and uses [`numpy.linalg.eig`](http://docs.scipy.org/doc/numpy-1.10.1/reference/generated/numpy.linalg.eig.html#numpy.linalg.eig) (mostly equivalent to [`scipy.linalg.eig`](http://docs.scipy.org/doc/scipy-0.17.0/reference/generated/scipy.linalg.eig.html)) to calculate the eigenvalues and -vectors.
* `eigh`: Calculates the covariance matrix and uses [`numpy.linalg.eigh`](http://docs.scipy.org/doc/numpy-1.10.1/reference/generated/numpy.linalg.eigh.html#numpy.linalg.eigh) to calculate the eigenvalues and -vectors.
* `svd`: Uses [`numpy.linalg.svd`](http://docs.scipy.org/doc/numpy-1.10.1/reference/generated/numpy.linalg.svd.html#numpy.linalg.svd) to calculate the eigenvalues and -vectors.
* `PCA`: Application of [`sklearn.decomposition.PCA`](http://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html). Internally centers the data and uses svd.
* `IncrementalPCA`: Application of [`sklearn.decomposition.IncrementalPCA`](http://scikit-learn.org/stable/modules/generated/sklearn.decomposition.IncrementalPCA.html). Internally centers the data and uses svd.
* `Eigenvalues`: Plots a comparison of all resulting eigenvalues.

Note that there are even more interesting methods to calculate principal components or for example independent components. An overview of algorithms can be found in the [scikit-learn user guide on decomposition](http://scikit-learn.org/stable/modules/decomposition.html).

In [ ]:
%matplotlib notebook
from collections import OrderedDict

import numpy as np
import sklearn.decomposition
import matplotlib.pyplot as plt

# Enables text output.
TEXTOUT = False 

# Choose a data set.
# data = np.array([[3,2,-1],[4,1,2],[4,2,9],[2,-3,1]])
data = np.array([[5, 2], [3, 4], [6, 1]])
# data = np.loadtxt('cars.csv', delimiter=',')


# Calculates normalization.
mean = np.mean(data, 0)
std = np.std(data, 0)

eigval = OrderedDict({'Raw Data': {}, 'Centered Data': {}, 'Normalized Data': {}})
eigvec = OrderedDict({'Raw Data': {}, 'Centered Data': {}, 'Normalized Data': {}})

data = OrderedDict({'Raw Data': data, 'Centered Data': data - mean, 'Normalized Data': (data - mean) / std})

def pca_eig(data):
    """Uses numpy.linalg.eig to calculate the PCA."""
    data = data.T @ data
    val, vec = np.linalg.eig(data)
    sort = np.argsort(val)
    return val[sort], vec[:,sort]

def pca_eigh(data):
    """Uses numpy.linalg.eigh to calculate the PCA."""
    data = data.T @ data
    val, vec = np.linalg.eigh(data)
    sort = np.argsort(val)
    return val[sort], vec[:,sort]

def pca_svd(data):
    """Uses numpy.linalg.svd to calculate the PCA."""
    u, s, v = np.linalg.svd(data)
    val, vec = s ** 2, v
    sort = np.argsort(val)
    return val[sort], vec[:,sort]
    
def pca_PCA(data):
    """Uses sklearn.decomposition.PCA to calculate the PCA."""
    pca = sklearn.decomposition.PCA().fit(data)
    val, vec = pca.explained_variance_, pca.components_
    sort = np.argsort(val)
    return val[sort], vec[:,sort]

def pca_IncrPCA(data):
    """Uses sklearn.decomposition.IncrementalPCA to calculate the PCA."""
    pca = sklearn.decomposition.IncrementalPCA().fit(data)
    val, vec = pca.explained_variance_, pca.components_
    sort = np.argsort(val)
    return val[sort], vec[:,sort]

pcas = {
    'eig': pca_eig,
    'eigh': pca_eigh,
    'svd': pca_svd,
    'PCA': pca_PCA,
    'IncrementalPCA': pca_IncrPCA,
}

for k, v in data.items():
    figure = plt.figure(k, figsize=(10,8))
    i = 1
    for name, pca in pcas.items():
        eigvalues, eigvecs = pca(v)
        eigval[k][name] = eigvalues
        eigvec[k][name] = eigvecs
        transformed = v @ eigvecs[:,0:2]
    
        figure = plt.figure(k)
        plt.subplot(2, 3, i)
        plt.gca().set_title(name)
        plt.scatter(*zip(*transformed))
        plt.subplot(2, 3, 6)
        plt.plot(sorted(eigval[k][name]), linestyle='None', marker='o+xd.'[i-1], label=name)
        plt.legend(loc=2, prop={'size':6})
        i += 1
    plt.gca().set_title('Eigenvalues')

if TEXTOUT:
    for k, v in data.items():
        print(k)
        for name in pcas.keys():
            print('\t', name)
            print('\t\tEigenvalues:')
            print('\t\t', eigval[k][name])
            print('\t\tEigenvectors:')
            print('\t\t', eigvec[k][name])